<a href="https://colab.research.google.com/github/mashdas/rapido_classical/blob/master/prune_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!unzip ct_rr.csv.zip

Archive:  ct_rr.csv.zip
  inflating: ct_rr.csv               
   creating: __MACOSX/
  inflating: __MACOSX/._ct_rr.csv    


In [ ]:
import pandas as pd
import numpy as np
from dateutil import parser

In [170]:
data=pd.read_csv('ct_rr.csv',na_values=[' None'])

In [171]:
len(data)

8381556

In [172]:
##Dropping the NA values
data.dropna(axis=0,inplace=True)

In [173]:
#Sanity Check
data.isnull().sum()

ts          0
number      0
pick_lat    0
pick_lng    0
drop_lat    0
drop_lng    0
dtype: int64

In [174]:
data.head(5)

,ts,number,pick_lat,pick_lng,drop_lat,drop_lng
0,2018-04-07 07:07:17,14626.0,12.313621,76.658195,12.287301,76.602280
1,2018-04-07 07:32:27,85490.0,12.943947,77.560745,12.954014,77.543770
2,2018-04-07 07:36:44,5408.0,12.899603,77.587300,12.934780,77.569950
3,2018-04-07 07:38:00,58940.0,12.918229,77.607544,12.968971,77.636375
4,2018-04-07 07:39:29,5408.0,12.899490,77.587270,12.934780,77.569950


In [176]:
len(data["number"].unique())

94274

In [177]:
##need to extract date and time from ts for more granular analysis

In [178]:
data["day"]=data.apply(lambda row:parser.parse(row['ts']).isoweekday(), axis = 1)

In [179]:
##Mapping days to values:
data["day"]=data["day"].map({1:"Mon",2:"Tue",3:"Wed",4:"Thr",5:"Fri",6:"Sat",7:"Sun"})

In [180]:
data.head(5)

,ts,number,pick_lat,pick_lng,drop_lat,drop_lng,day
0,2018-04-07 07:07:17,14626.0,12.313621,76.658195,12.287301,76.602280,Sat
1,2018-04-07 07:32:27,85490.0,12.943947,77.560745,12.954014,77.543770,Sat
2,2018-04-07 07:36:44,5408.0,12.899603,77.587300,12.934780,77.569950,Sat
3,2018-04-07 07:38:00,58940.0,12.918229,77.607544,12.968971,77.636375,Sat
4,2018-04-07 07:39:29,5408.0,12.899490,77.587270,12.934780,77.569950,Sat


In [181]:
def hour_mapper(x):
  stuff=x.split(' ')[1].split(':')
  return int(stuff[0])+int(stuff[1])/60+int(stuff[2])/3600


In [182]:
data["time"]=data.apply(lambda row: hour_mapper(row['ts']),axis=1)

In [183]:
data.head()

,ts,number,pick_lat,pick_lng,drop_lat,drop_lng,day,time
0,2018-04-07 07:07:17,14626.0,12.313621,76.658195,12.287301,76.602280,Sat,7.121389
1,2018-04-07 07:32:27,85490.0,12.943947,77.560745,12.954014,77.543770,Sat,7.540833
2,2018-04-07 07:36:44,5408.0,12.899603,77.587300,12.934780,77.569950,Sat,7.612222
3,2018-04-07 07:38:00,58940.0,12.918229,77.607544,12.968971,77.636375,Sat,7.633333
4,2018-04-07 07:39:29,5408.0,12.899490,77.587270,12.934780,77.569950,Sat,7.658056


In [184]:
len(data["number"].unique())

94274

In [185]:
d_mon,d_tue,d_wed,d_thr,d_fri,d_sat,d_sun=data[data["day"]=="Mon"].copy(),data[data["day"]=="Tue"].copy(),data[data["day"]=="Wed"].copy(),data[data["day"]=="Thr"].copy(),data[data["day"]=="Fri"].copy(),data[data["day"]=="Sat"].copy(),data[data["day"]=="Sun"].copy()

In [186]:
set(data["day"])

{'Fri', 'Mon', 'Sat', 'Sun', 'Thr', 'Tue', 'Wed'}

In [187]:
d_mon.head()

,ts,number,pick_lat,pick_lng,drop_lat,drop_lng,day,time
15091,2018-04-09 00:10:58,29372.0,12.925840,77.60910,13.002063,77.622795,Mon,0.182778
15092,2018-04-09 00:11:52,23598.0,12.936879,77.60243,12.887472,77.592440,Mon,0.197778
15093,2018-04-09 00:12:10,29372.0,12.925840,77.60910,13.002063,77.622795,Mon,0.202778
15094,2018-04-09 00:12:28,23598.0,12.936879,77.60243,12.887472,77.592440,Mon,0.207778
15095,2018-04-09 00:13:12,23598.0,12.936879,77.60243,12.887472,77.592440,Mon,0.220000


In [188]:
def eligible(stuff):
  tups=[]
  ids=stuff["number"].unique()
  for x in ids:
    tups.append((x,stuff[stuff["number"]==x].shape[0]))
  tups.sort(key=lambda x:x[1],reverse=True)
  filtered_tups=filter(lambda x:x[1]>30,tups)
  f_list=[x[0] for x in list(filtered_tups)]
  return stuff.loc[stuff["number"].isin(f_list)].copy()  

In [189]:
def data_beautify(s):
  s.drop(["ts","day","drop_lat","drop_lng"],inplace=True,axis=1)
  s["result"]=1
  return s

In [191]:
sets=[d_mon,d_tue,d_wed,d_thr,d_fri,d_sat,d_sun]

In [192]:
i=1
for x in sets:
  data_beautify(eligible(x.copy())).to_csv(str(i)+'data.csv')
  i=i+1